读取modified_tasks.json

然后整理成Pick up the pyramid-shaped sachet and place it into the box , sachet position relative to gripper is (-0.101, 0.158, -0.103)这样的形式，其中relative position用原来的值算出来

最后保存成relative_task.json

In [2]:
import json

# 读取原始的 modified_tasks.json 文件
with open('modified_tasks.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

# 存储转换后的数据
relative_data = []

for entry in data:
    gripper = entry['task'].split("gripper at")[1].split("),")[0].strip(" (") + ")"
    sachet = entry['task'].split("sachet at")[1].strip()

    # 解析坐标字符串成元组
    gripper_pos = tuple(map(float, gripper.strip("()").split(",")))
    sachet_pos = tuple(map(float, sachet.strip("()").split(",")))

    # 计算相对位置：sachet - gripper
    relative_pos = tuple(round(s - g, 3) for s, g in zip(sachet_pos, gripper_pos))

    # 构建新的 task 字符串
    task = f"Pick up the pyramid-shaped sachet and place it into the box, sachet position relative to gripper is {relative_pos}"

    # 构建新的字典条目
    relative_entry = {
        "episode_index": entry["episode_index"],
        "frame_index": entry["frame_index"],
        "task": task
    }

    relative_data.append(relative_entry)

# 写入新的 relative_task.json 文件
with open('relative_task.json', 'w') as f:
    for entry in relative_data:
        json.dump(entry, f)
        f.write('\n')


防止识别不了太精细，网格化，单位3cm


In [3]:
import json

GRID_SIZE = 0.05  # 5cm = 0.05m

# 读取原始的 modified_tasks.json 文件
with open('modified_tasks.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

relative_data = []

for entry in data:
    gripper = entry['task'].split("gripper at")[1].split("),")[0].strip(" (") + ")"
    sachet = entry['task'].split("sachet at")[1].strip()

    # 解析坐标
    gripper_pos = tuple(map(float, gripper.strip("()").split(",")))
    sachet_pos = tuple(map(float, sachet.strip("()").split(",")))

    # 计算相对位置（以grid为单位）
    relative_pos = tuple(int(round((s - g) / GRID_SIZE)) for s, g in zip(sachet_pos, gripper_pos))

    # 生成新的 task 描述
    task = f"Pick up the pyramid-shaped sachet and place it into the box, sachet position relative to gripper is {relative_pos} in 5cm grid units."

    # 创建新 entry
    relative_entry = {
        "episode_index": entry["episode_index"],
        "frame_index": entry["frame_index"],
        "task": task
    }

    relative_data.append(relative_entry)

# 保存为 relative_task.json
with open('relative_task_grid.json', 'w') as f:
    for entry in relative_data:
        json.dump(entry, f)
        f.write('\n')


结构化
"""
task:
objects
"""
这样的

In [ ]:
import json

input_file = "modified_tasks.jsonl"
output_file = "mtask_structural.jsonl"

with open(input_file, "r") as fin, open(output_file, "w") as fout:
    for line in fin:
        data = json.loads(line)

        # 原始 task 字符串
        task_str = data["task"]

        # 拆分任务描述和坐标部分
        task_desc, coords_str = task_str.split("|", 1)
        task_desc = task_desc.strip().capitalize()  # 首字母大写
        coords_str = coords_str.strip()

        # 提取 gripper 和 sachet 坐标
        import re
        gripper_match = re.search(r"gripper at \(([^)]+)\)", coords_str)
        sachet_match = re.search(r"sachet at \(([^)]+)\)", coords_str)

        if not gripper_match or not sachet_match:
            raise ValueError(f"无法解析坐标: {coords_str}")

        gripper_pos = tuple(map(float, gripper_match.group(1).split(",")))
        sachet_pos = tuple(map(float, sachet_match.group(1).split(",")))

        # 计算相对坐标
        rel_pos = tuple(s - g for s, g in zip(sachet_pos, gripper_pos))

        # 生成结构化任务描述
        structured_task = f"""task: {task_desc}
objects:
    name: sachet
    position_relative_to_gripper:({rel_pos[0]:.3f}, {rel_pos[1]:.3f}, {rel_pos[2]:.3f})
    unit: meters"""

        # 写入新文件
        out_data = {
            "episode_index": data["episode_index"],
            "frame_index": data["frame_index"],
            "task": structured_task
        }
        fout.write(json.dumps(out_data, ensure_ascii=False) + "\n")

print(f"已保存到 {output_file}")
